### Import

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import keras

### Setting

In [ ]:
img_width, img_height = 256, 256
train_data_dir = "training_set"
validation_data_dir = "validation_set"
nb_train_samples = 1000
nb_validation_samples = 200 
batch_size = 50
epochs = 50

### Using VGG19 to do transfer learning
#### Creat a fully connected layer to classfy the object

In [ ]:
# include top = the final level( fully connected )
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

# Lock every layer in the VGG19( because the layers are already trained by imagenet ) 
for layer in model.layers:
    layer.trainable = False

# creat a fully connected layer
x = model.output
x = Flatten()(x) # fully connected
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x) # avoid over fitting
x = Dense(1024, activation="relu")(x)
# to classfy two labels
predictions = Dense( 2, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])


### The Data Setting ( training/testing data )

In [ ]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30 )

validation_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = validation_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")


### Set ealy stop to avoid the overfitting

In [ ]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("my_model.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=10)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=100, verbose=1, mode='auto')
tb = keras.callbacks.TensorBoard( log_dir = './log' )

### Train the model
#### Using tensorboard to check the loss and acc

In [ ]:
# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = 5000,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early, tb])